In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import librosa
import os
import random
import seaborn as sns
import pandas as pd
import pickle
import math
import torchaudio
import csv
import string

from torch.utils.data import ConcatDataset
from torchaudio.transforms import TimeMasking, FrequencyMasking
from torchvision.ops import SqueezeExcitation
from torchinfo import summary
from torchsummary import summary
from torchvision.ops import SqueezeExcitation
from tqdm import tqdm
from torchvision import datasets, transforms
from collections import defaultdict
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import DataLoader, Dataset
from scipy.io import wavfile
from functools import reduce
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option('future.no_silent_downcasting', True)

### Loading data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.n_samples = 0
        self.dataset = []
        self.labels = set()  # To track unique labels
        self.load_audio_files(self.data_dir)

    def load_audio_files(self, path: str):
        for dirname, _, filenames in os.walk(path):
            for filename in filenames:
                file_path = os.path.join(dirname, filename)
                
                # label = dirname.split('/')[-1]  # on MAC 
                label = os.path.basename(dirname)   # on Windows
       
                # my implementation start
                if '0' <= label <= '9':
                    label_index = ord(label) - ord('0')
                elif 'a' <= label <= 'z':
                    label_index = ord(label) - ord('a') + 10
                else:
                    raise ValueError(f"Unexpected label: {label}")
                    break
                label_tensor = torch.tensor(label_index)
                # my implementation done
                
                # Add the label to the set of unique labels
                self.labels.add(label_tensor.item())
                
                # Load audio
                waveform, sample_rate = torchaudio.load(file_path)
                if self.transform is not None:
                    waveform_transformed = self.transform(waveform)
                
                if waveform_transformed.shape[2] != 64:
                    print("Wrong shape:", waveform_transformed.shape)
                    continue
                
                self.n_samples += 1
                self.dataset.append((waveform, label_tensor))

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        waveform, label = self.dataset[idx]
        return waveform, label

    def num_classes(self):
        return len(self.labels)  # Return the number of unique labels

In [ ]:
# def add_noise(waveform, noise_factor=None):
#     """
#     Adds random noise to the waveform.
    
#     Args:
#         waveform (Tensor): Audio waveform tensor.
#         noise_factor (float): Scaling factor for the noise.
        
#     Returns:
#         Tensor: Noisy waveform.
#     """
#     noise = torch.randn_like(waveform)
#     noisy_waveform = waveform + noise_factor * noise
#     return noisy_waveform

def add_noise(stroke, noise_factor=None):
    """
    Adds random noise to an audio stroke tensor.
    
    Args:
    - stroke (torch.Tensor): The input audio stroke tensor.
    - noise_factor (float): The factor determining the noise level relative to the stroke's signal.

    Returns:
    - torch.Tensor: The noisy audio stroke tensor.
    """
    # Generate random noise
    noise = torch.randn(stroke.size()) * noise_factor

    # Calculate the standard deviation of the stroke signal
    std_dev = stroke.std()

    # Scale the noise level based on the signal's standard deviation
    noise = noise * std_dev

    # Add noise to the original stroke
    noisy_stroke = stroke + noise
    
    return noisy_stroke

def add_noise_and_save(input_dir, output_dir, noise_factor=None):
    """
    Traverses the input directory, adds noise to each .wav file, and saves
    the noisy audio to the output directory while maintaining folder structure.
    
    Args:
        input_dir (str): Path to the directory containing original .wav files.
        output_dir (str): Path where the noisy audio files will be saved.
        noise_factor (float): Factor controlling the amount of noise.
    """
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.wav'):
                # Construct the full file path and load the audio
                file_path = os.path.join(root, file)
                waveform, sample_rate = torchaudio.load(file_path)
                
                # Add noise to the waveform
                noisy_waveform = add_noise(waveform, noise_factor)
                
                # Recreate the subdirectory structure in the output folder
                relative_path = os.path.relpath(root, input_dir)
                new_dir = os.path.join(output_dir, relative_path)
                os.makedirs(new_dir, exist_ok=True)
                # print(new_dir)
                
                # Define the new file path and save the noisy audio
                new_file_path = os.path.join(new_dir, file)
                # print(new_file_path)
                torchaudio.save(new_file_path, noisy_waveform, sample_rate)
                # print(f"Saved noisy file to {new_file_path}")

# Define your input and output directories
input_dir = '../../new_dataset_phone'          # Replace with your original dataset folder
output_dir = '../../noise_audio_dataset_phone'   # New folder for the noisy audio files

# Generate and save the noisy audio files
add_noise_and_save(input_dir, output_dir, noise_factor=0.06)


In [ ]:
sample_rate = 44100
to_mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate, n_mels=64, hop_length=300, n_fft=2048, win_length=1024)
mel_spectrogram_to_numpy = lambda spectrogram: spectrogram.log2()[0,:,:].numpy()
transforms = Compose([to_mel_spectrogram, mel_spectrogram_to_numpy, ToTensor()])
dataset = AudioDataset('../../noise_audio_dataset_phone', transforms)
print("number of classes:", dataset.num_classes())

In [ ]:
from sklearn.model_selection import train_test_split
import torch

# Assume your dataset has a 'targets' attribute or you can extract labels from it
targets = [data[1] for data in dataset]  # Assuming dataset returns (data, label) pairs

# Split the dataset indices with stratification
train_indices, tmp_indices = train_test_split(
    range(len(dataset)), 
    test_size=0.3,  # 30% of the data goes to val+test
    stratify=targets
)

val_indices, test_indices = train_test_split(
    tmp_indices, 
    test_size=0.33,  # 33% of the 30% goes to the test set, i.e., 10% of the original dataset
    stratify=[targets[i] for i in tmp_indices]
)

# Create subsets of the dataset based on the indices
init_train_set = torch.utils.data.Subset(dataset, train_indices)
init_val_set = torch.utils.data.Subset(dataset, val_indices)
init_test_set = torch.utils.data.Subset(dataset, test_indices)

# Print the sizes for verification
print("Sample rate:", sample_rate)
print(f"Train set size: {len(init_train_set)}, Validation set size: {len(init_val_set)}, Test set size: {len(init_test_set)}")


### Data augmentation setup

In [ ]:
class TrainingDataset(Dataset):
    def __init__(self, base_dataset, transformations):
        super(TrainingDataset, self).__init__()
        self.base = base_dataset
        self.transformations = transformations

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        waveform, label = self.base[idx]
        return self.transformations(waveform), label

In [ ]:
class TimeShifting():
    def __call__(self, samples):
        samples = samples.numpy()        
        shift = int(samples.shape[1] * 0.3)
        random_shift = random.randint(0, shift)
        data_roll = np.zeros_like(samples)
        data_roll[0] = np.roll(samples[0], random_shift)
        data_roll[1] = np.roll(samples[1], random_shift)
        return torch.tensor(data_roll)

In [ ]:
aug_transforms = Compose([
    TimeShifting(),
    to_mel_spectrogram, mel_spectrogram_to_numpy, ToTensor(),
    FrequencyMasking(7),
    TimeMasking(7),
    FrequencyMasking(7),
    TimeMasking(7)
])

train_set = TrainingDataset(init_train_set, aug_transforms)
train_set_no_aug = TrainingDataset(init_train_set, transforms)
val_set = TrainingDataset(init_val_set, transforms)
test_set = TrainingDataset(init_test_set, transforms)

### CoAtNet (Model)

In [ ]:
class Stem(nn.Sequential):
    def __init__(self, out_channels):
        super().__init__(
            nn.Conv2d(1, out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3)
        )

In [ ]:
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, expansion_factor=4):
        super().__init__()
        self.mb_conv = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, in_channels * expansion_factor, kernel_size=1),
            nn.BatchNorm2d(in_channels * expansion_factor),
            nn.GELU(),
            nn.Conv2d(in_channels * expansion_factor, in_channels * expansion_factor, kernel_size=3, padding=1, groups=in_channels * expansion_factor),
            nn.BatchNorm2d(in_channels * expansion_factor),
            nn.GELU(),
            SqueezeExcitation(in_channels * expansion_factor, in_channels, activation=nn.GELU),
            nn.Conv2d(in_channels * expansion_factor, out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels)
        )
    
    def forward(self, x):
        return x + self.mb_conv(x)

In [ ]:
class DownsamplingMBConv(MBConv):
    def __init__(self, in_channels, out_channels, expansion_factor=4):
        super().__init__(in_channels, out_channels, expansion_factor=4)
        self.mb_conv[1] = nn.Conv2d(in_channels, in_channels * expansion_factor, kernel_size=1, stride = 2)
        self.channel_projection = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    
    def forward(self, x):
        return self.channel_projection(self.pool(x)) + self.mb_conv(x)

In [ ]:
class RelativeAttention2d(nn.Module):
    def __init__(self, in_channels, out_channels, image_size, heads=8, head_size=32):
        super().__init__()
        heads = out_channels // head_size
        self.heads = heads
        self.head_size = head_size
        self.image_size = image_size
        self.head_dim = heads * head_size
        self.attend = nn.Softmax(dim=-2) # Taken from My_CoAtNet
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.to_q = nn.Linear(in_channels, self.head_dim)
        self.to_k = nn.Linear(in_channels, self.head_dim)
        self.to_v = nn.Linear(in_channels, self.head_dim)
        self.to_output = nn.Sequential(
            nn.Linear(self.head_dim, out_channels),
            nn.Dropout(0.3)
        )
        self.normalization = nn.LayerNorm(in_channels)
        
        self.relative_bias = nn.Parameter(torch.randn(heads, (2 * image_size - 1) * (2 * image_size - 1)))
        self.register_buffer("relative_indices", self.get_indices(image_size, image_size)) # Taken from My_CoAtNet
        self.precomputed_relative_bias = None
    
    def norm(self, x):
        x = x.transpose(1, -1) # Taken from My_CoAtNet
        x = self.normalization(x)
        x = x.transpose(-1, 1) # Taken from My_CoAtNet
        return x
    
    def get_relative_biases(self):
        # Relative bias caching mentioned in CoAtNet: Marrying Convolution and Attention for All Data Sizes
        if not self.training:
            return self.precomputed_relative_bias
        # Taken from od My_CoAtNet
        indices = self.relative_indices.expand(self.heads, -1)
        rel_pos_enc = self.relative_bias.gather(-1, indices)
        rel_pos_enc = rel_pos_enc.unflatten(-1, (self.image_size * self.image_size, self.image_size * self.image_size))
        return rel_pos_enc
    
    def reshape_for_linear(self, x):
        b, _, _, _ = x.shape
        return x.reshape(b, self.image_size * self.image_size, self.in_channels)
    
    def attention_score(self, x):
        b, _, h, _ = x.shape
        q = self.to_q(self.reshape_for_linear(x)).view(b, self.heads, self.head_size, -1) # Taken from My_CoAtNet
        k = self.to_k(self.reshape_for_linear(x)).view(b, self.heads, self.head_size, -1) # Taken from My_CoAtNet
        dots = torch.matmul(k.transpose(-1, -2), q) / math.sqrt(self.head_dim)
        relative_biases_indexed = self.get_relative_biases()
        return self.attend(dots + relative_biases_indexed)
    
    def relative_attention(self, x):
        b, _, _, _ = x.shape
        v = self.to_v(self.reshape_for_linear(x)).view(b, self.heads, self.head_size, -1) # Taken from My_CoAtNet
        out = torch.matmul(v, self.attention_score(x)) # I figured this out after debugging (Still the same as My_CoAtNet)
        out = out.view(b, self.image_size, self.image_size, -1)
        return self.to_output(out).view(b, self.out_channels, self.image_size, self.image_size)
        
    def forward(self, x):
        return x + self.relative_attention(self.norm(x))
    
    def train(self, training):
        if not training:
            self.precomputed_relative_bias = self.get_relative_biases()
        super().train(training)
        
    # Taken from My_CoAtNet
    @staticmethod
    def get_indices(h, w):
        y = torch.arange(h, dtype=torch.long)
        x = torch.arange(w, dtype=torch.long)
        
        y1, x1, y2, x2 = torch.meshgrid(y, x, y, x)
        indices = (y1 - y2 + h - 1) * (2 * w - 1) + x1 - x2 + w - 1
        indices = indices.flatten()
        
        return indices
        

In [ ]:
class DownsamplingRelativeAttention2d(RelativeAttention2d):
    def __init__(self, in_channels, out_channels, image_size, heads=8, head_size=32):
        super().__init__(in_channels, out_channels, image_size, heads=8, head_size=32)
        self.channel_projection = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.normalization = nn.LayerNorm(in_channels)


    def norm(self, x):
        x = x.transpose(1, -1) # Taken from My_CoAtNet
        x = self.normalization(x)
        x = x.transpose(-1, 1) # Taken from My_CoAtNet
        return x
        
    def forward(self, x):
        return self.channel_projection(self.pool(x)) + self.relative_attention(self.pool(self.norm(x)))

In [ ]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, out_channels, expansion_factor=4):
        super().__init__()
        hidden_dim = out_channels * expansion_factor
        self.ffn = nn.Sequential(
            nn.Linear(out_channels, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, out_channels),
            nn.Dropout(0.3)
        )
        self.normalization = nn.LayerNorm(out_channels)
        self.out_channels = out_channels

    
    def norm(self, x):
        x = x.transpose(1, -1) # Taken from My_CoAtNet
        x = self.normalization(x)
        x = x.transpose(-1, 1) # Taken from My_CoAtNet
        return x
    
    def forward(self, x):
        old_shape = x.shape
        batch_size = old_shape[0]
        return x + torch.reshape(self.ffn(torch.reshape(self.norm(x), (batch_size, -1, self.out_channels))), old_shape)
    

In [ ]:
class DownsampleTransformerBlock(nn.Sequential):
    def __init__(self, in_channels, out_channels, image_size):
        attention = DownsamplingRelativeAttention2d(in_channels, out_channels, image_size)
        ffn = FeedForwardNetwork(out_channels)
        super().__init__(
            attention,
            ffn
        )

In [ ]:
class TransformerBlock(nn.Sequential):
    def __init__(self, in_channels, out_channels, image_size):
        attention = RelativeAttention2d(in_channels, out_channels, image_size)
        ffn = FeedForwardNetwork(out_channels)
        super().__init__(
            attention,
            ffn
        )

In [ ]:
class Head(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(in_channels, num_classes)
        self.in_channels = in_channels
        
    def forward(self, x):
        batch_size = x.shape[0]
        x = self.pool(x)
        x = torch.reshape(x, (batch_size, -1, self.in_channels))
        return torch.squeeze(self.fc(x))
    

In [ ]:
class MyCoAtNet(nn.Sequential):
    def __init__(self, nums_blocks, layer_out_channels, num_classes=36):
        s0 = nn.Sequential(Stem(layer_out_channels[0]))
        
        s1 = [DownsamplingMBConv(layer_out_channels[0], layer_out_channels[1])]
        for i in range(nums_blocks[1] - 1):
            s1.append(MBConv(layer_out_channels[1], layer_out_channels[1]))
        s1 = nn.Sequential(*s1)
        
        s2 = [DownsamplingMBConv(layer_out_channels[1], layer_out_channels[2])]
        for i in range(nums_blocks[2] - 1):
            s2.append(MBConv(layer_out_channels[2], layer_out_channels[2]))
        s2 = nn.Sequential(*s2)
        
        s3 = [DownsampleTransformerBlock(layer_out_channels[2], layer_out_channels[3], 64 // 16)]
        for i in range(nums_blocks[3] - 1):
            s3.append(TransformerBlock(layer_out_channels[3], layer_out_channels[3], 64 // 16))
        s3 = nn.Sequential(*s3)
        
        s4 = [DownsampleTransformerBlock(layer_out_channels[3], layer_out_channels[4], 64 // 32)]
        for i in range(nums_blocks[4] - 1):
            s4.append(TransformerBlock(layer_out_channels[4], layer_out_channels[4], 64 // 32))
        s4 = nn.Sequential(*s4)
        
        head = Head(layer_out_channels[4], num_classes)
        
        super().__init__(
            s0,
            s1,
            s2,
            s3,
            s4,
            head
        )
            

In [ ]:
# CoAtNet-1
nums_blocks = [2, 2, 3, 5, 2]           # L
channels = [64, 96, 192, 384, 768]      # D

model = MyCoAtNet(nums_blocks, channels)

In [ ]:
def init_linear(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None: nn.init.zeros_(m.bias)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.apply(init_linear)
model.to(device)

print("Number of parameters: {:,}".format(sum(p.numel() for p in model.parameters())))

summary(model, input_size=(1, 64, 64))

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.load_state_dict(torch.load("CoAtNet-1-Best-Phone.pkl"))
model.to(device)
model.eval()

### Training

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_set,
    batch_size=16,
    shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    val_set,
    batch_size=16,
    shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(
    test_set,
    batch_size=16,
    shuffle=True
)

In [ ]:
# Adapted from My_CoAtNet
def separate_parameters(model):
    parameters_decay = set()
    parameters_no_decay = set()
    modules_weight_decay = (nn.Linear, nn.Conv2d)
    modules_no_weight_decay = (nn.LayerNorm, nn.BatchNorm2d)
    
    for m_name, m in model.named_modules():
        for param_name, param in m.named_parameters():
            full_param_name = f"{m_name}.{param_name}" if m_name else param_name

            if isinstance(m, modules_no_weight_decay):
                parameters_no_decay.add(full_param_name)
            elif param_name.endswith("bias"):
                parameters_no_decay.add(full_param_name)
            elif isinstance(m, modules_weight_decay):
                parameters_decay.add(full_param_name)
    
    # sanity check
    assert len(parameters_decay & parameters_no_decay) == 0
    assert len(parameters_decay) + len(parameters_no_decay) == len(list(model.parameters()))

    return parameters_decay, parameters_no_decay

In [ ]:
cost = torch.nn.CrossEntropyLoss()

learning_rate = 5e-4
weight_decay = 0.1
num_epochs = 1100
min_learning_rate = 1e-6

param_dict = {pn: p for pn, p in model.named_parameters()}
parameters_decay, parameters_no_decay = separate_parameters(model)
parameter_groups = [
    {"params": [param_dict[pn] for pn in parameters_decay], "weight_decay": weight_decay},
    {"params": [param_dict[pn] for pn in parameters_no_decay], "weight_decay": 0.0},
]
optimizer = torch.optim.Adam(parameter_groups, lr=learning_rate)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=min_learning_rate / learning_rate, total_iters=num_epochs)

In [ ]:
# Adapted from My_CoAtNet
def plot_results(train_losses, train_accuracies, val_losses, val_accuracies):
    plt.figure(figsize=(12, 4))
    plt.subplot(1,2,1)
    plt.title("Loss")
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.legend(["Training loss", "Validation loss"], loc =  "best")
    plt.subplot(1,2,2)
    plt.title("Accuracy")
    plt.plot(train_accuracies)
    plt.plot(val_accuracies)
    plt.legend(["Training accuracy", "Validation accuracy"], loc =  "best")
    plt.show()

In [ ]:
def save_model(model, path):
    torch.save(model.state_dict(), path)

In [ ]:
best_model_path = "CoAtNet-1-Best-Phone.pkl"
model_path = "CoAtNet-1-Phone.pkl"

In [ ]:
model.load_state_dict(torch.load(best_model_path))
model.to(device)
model.eval()

### LLM Part

In [ ]:
# Create mappings from indices to syllables and vice versa
digits_and_syllables = list('0123456789abcdefghijklmnopqrstuvwxyz')
idx_to_syllable = {idx: syllable for idx, syllable in enumerate(digits_and_syllables)}
syllable_to_idx = {syllable: idx for idx, syllable in idx_to_syllable.items()}

In [ ]:
# Combine test and validation datasets
combined_dataset = ConcatDataset([val_set, test_set])

In [ ]:
# Create a mapping from syllables to dataset indices
syllable_to_indices = {}
for idx in range(len(combined_dataset)):
    _, label = combined_dataset[idx]
    label = label.item() 
    syllable = idx_to_syllable[label]
    if syllable not in syllable_to_indices:
        syllable_to_indices[syllable] = []
    syllable_to_indices[syllable].append(idx)

In [ ]:
# Function to process sentences into syllables
def get_syllables(sentence):
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    # sentence = sentence.replace(' ',  '').lower()
    syllables = list(sentence)
    return syllables

In [ ]:
# Read sentences from the text file
with open('../sentences/ft_sentences.txt', 'r') as f:
    sentences = f.readlines()

print("Number of sentences:", len(sentences))

In [ ]:
for i in range(1, 12308):
    if 12308 % i == 0:
        print(i)

In [ ]:
import difflib

def compute_accuracy_and_wrong_syllables(true_sentence, predicted_sentence):
    # Character-level accuracy using SequenceMatcher
    char_matcher = difflib.SequenceMatcher(None, true_sentence, predicted_sentence)
    accuracy = char_matcher.ratio()
    
    # Word-level wrong syllable count using SequenceMatcher on word lists
    true_words = true_sentence.split()
    predicted_words = predicted_sentence.split()
    word_matcher = difflib.SequenceMatcher(None, true_words, predicted_words)
    
    # Calculate wrong syllables based on insert, delete, and replace operations
    wrong_syllables = sum(1 for tag, _, _, _, _ in word_matcher.get_opcodes() if tag in ('insert', 'delete', 'replace'))
    
    return accuracy, wrong_syllables


In [ ]:
# Process each sentence
# noise_factor = 0
# results = []

# for sentence in tqdm(sentences[:16]):
#     sentence = sentence.strip()
#     syllables = get_syllables(sentence)
#     true_sentence = ''.join(syllables)
#     predicted_syllables = []

#     for syllable in syllables:
#         if syllable in syllable_to_indices:
#             # Randomly select an index for the syllable
#             idx = random.choice(syllable_to_indices[syllable])
#             # Retrieve the image and label from the dataset
#             image, _ = combined_dataset[idx]

#             noise = torch.randn_like(image) * noise_factor
#             image = image + noise
#             image = image.unsqueeze(0).to(device)

#             with torch.no_grad():
#                 output = model(image)
#                 output = output.reshape(1, -1)
#                 _, predicted_idx = torch.max(output.data, 1)
#                 predicted_syllable = idx_to_syllable[predicted_idx.item()]
#         else:
#             if random.random() < 0.90:
#                 predicted_syllable = ' '
#             else:
#                 predicted_syllable = random.choice(digits_and_syllables)
#         predicted_syllables.append(predicted_syllable)
    
#     predicted_sentence = ''.join(predicted_syllables)
#     accuracy, wrong_syllables = compute_accuracy_and_wrong_syllables(true_sentence, predicted_sentence)
#     results.append([true_sentence, predicted_sentence, accuracy, wrong_syllables])
#     # print(f"accuracy: {accuracy}, wrong: {wrong_syllables}, true: {true_sentence}, predicted: {predicted_sentence}")

In [ ]:
# Process sentences in batches
batch_size = 17  # Adjust based on available GPU memory
noise_factor = 0
results = []

for i in tqdm(range(0, len(sentences[:]), batch_size)):
    batch_sentences = sentences[i : i + batch_size]
    batch_sentences = [sentence.strip() for sentence in batch_sentences]
    
    batch_true_sentences = [''.join(get_syllables(sentence)) for sentence in batch_sentences]
    batch_predicted_sentences = []

    for true_sentence in batch_true_sentences:
        syllables = list(true_sentence)
        batch_images = []
        batch_valid_indices = []

        # Collect syllables in a batch
        for syllable in syllables:
            if syllable in syllable_to_indices:
                idx = random.choice(syllable_to_indices[syllable])
                image, _ = combined_dataset[idx]
                noise = torch.randn_like(image) * noise_factor
                batch_images.append(image + noise)
                batch_valid_indices.append(len(batch_images) - 1)
            else:
                batch_images.append(None)
                batch_valid_indices.append(None)

        # Process valid images as a batch
        if any(img is not None for img in batch_images):
            valid_images = [img for img in batch_images if img is not None]
            valid_images = torch.stack(valid_images).to(device)

            with torch.no_grad():
                output = model(valid_images)
                _, predicted_indices = torch.max(output.data, 1)

            predicted_syllables = [''] * len(batch_images)
            valid_idx = 0
            for j, idx in enumerate(batch_valid_indices):
                if idx is not None:
                    predicted_syllables[j] = idx_to_syllable[predicted_indices[valid_idx].item()]
                    valid_idx += 1
                else:
                    predicted_syllables[j] = ' ' if random.random() < 0.90 else random.choice(digits_and_syllables)

            predicted_sentence = ''.join(predicted_syllables)
        else:
            print("oh no")
            predicted_sentence = ' ' * len(true_sentence)

        batch_predicted_sentences.append(predicted_sentence)

    # Compute accuracy and wrong syllables for the batch
    for true_sentence, predicted_sentence in zip(batch_true_sentences, batch_predicted_sentences):
        accuracy, wrong_syllables = compute_accuracy_and_wrong_syllables(true_sentence, predicted_sentence)
        results.append([true_sentence, predicted_sentence, accuracy, wrong_syllables])


In [ ]:
len(results)

In [ ]:
# accuracy average and wrong syllables sum
accuracy_avg = sum(result[2] for result in results) / len(results)
wrong_syllables_sum = sum(result[3] for result in results)

print("Noise factor:", noise_factor)
print(f"Average accuracy: {accuracy_avg:.4f}")
print(f"Total wrong syllables: {wrong_syllables_sum}")

In [ ]:
with open(f'results/ft_noise_0.06.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['True Sentence', 'Predicted Sentence', 'Accuracy', 'Wrong syllables'])
    for row in results:
        writer.writerow(row)

In [ ]:
# concatenate all the results
import pandas as pd
import glob

csv_files = glob.glob("results/ft_noise_*.csv")
df_list = []

for file in csv_files:
    noise_factor = float(file.split('_')[-1].replace('.csv', ''))
    df = pd.read_csv(file)
    df["Noise Factor"] = noise_factor
    df_list.append(df)

# Concatenate all dataframes into one
final_df = pd.concat(df_list, ignore_index=True)
# shuffle the dataframe
final_df = final_df.sample(frac=1).reset_index(drop=True)
# Save to a new CSV file
final_df.to_csv("results/ft_phone_ds.csv", index=False)

print("CSV files combined successfully.")
